# Rename files in S3 given an CSV file with old and new file names

This notebook is part of the Spyfish Aotearoa data cleaning effort.


In [ ]:
# Last change: 2025.11.22

In [ ]:
# DEV
# Uncomment, if you want to include local coding changes continuously.
# %load_ext autoreload
# %autoreload 2

In [ ]:
import pandas as pd
import os
import csv


from sftk.common import LOCAL_DATA_FOLDER_PATH, MOVIE_EXTENSIONS
from sftk.s3_handler import S3Handler

# Connect to S3

In [ ]:
s3_handler = S3Handler()

# Get S3 keys of movies of interest

In [ ]:
movies_in_s3 = s3_handler.get_file_paths_set_from_s3(prefix="media/BNP_2019", suffixes=".mp4") 
# ^ could also use MOVIE_EXTENSIONS for suffixes.

In [ ]:

output_csv_file = 'file_paths_per_row.csv'

with open(output_csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)

    # You might want a header to describe the data
    writer.writerow(["File Path"])

    # Iterate through the set and write each item as a new row
    # Note: Sets are unordered, so ordering it here to make it more readable.
    for item in sorted(list(movies_in_s3)):
        writer.writerow([item])

print(f"File paths saved to {output_csv_file}")

# Create the rename_movies.csv with the OLD and NEW name pairs

# Read the csv with old and new filenames (S3 keys)

Change variables here to match your use case 

In [ ]:
# define the path of the csv
file_path = os.path.join(LOCAL_DATA_FOLDER_PATH, "rename_movies.csv")

# Define the names of the columns containing the old names and the new names
new_name_column = "NEW"
old_name_column = "OLD"

In [ ]:
# Read the CSV file containing  into a DataFrame
rename_csv_df = pd.read_csv(file_path)

# Extract the dict with keys with old name and values with the new name
rename_pairs = dict(zip(rename_csv_df[old_name_column], rename_csv_df[new_name_column]))

# review the dictionary
print(len(rename_pairs))
rename_pairs

# Rename files in the bucket.

Try_run does not rename, but runs the code to see what will be affected, change to False when ready.

In [ ]:
s3_handler = S3Handler()
s3_handler.rename_s3_objects_from_dict(rename_pairs, suffixes=MOVIE_EXTENSIONS, try_run=False)

In [ ]:

# End